In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchvision import models

Task1 
1. Load CIFAR1-10 dataset

In [2]:
# 1. Load CIFAR-10 Dataset
transform = transforms.ToTensor()
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=64, shuffle=True)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=64, shuffle=False)

100.0%


In [5]:
class SimpleCNN(nn.Module):
    def __init__(self, activation_fn=nn.LeakyReLU()):
        super(SimpleCNN, self).__init__()
        # The first layer has 3 input channels and 32 output channels. The point here is
        # that the CIFR images are 32 * 32, and RGB channels after the to tensor. So, we take
        self.conv_layer = nn.Conv2d(3, 32, kernel_size=5) 
        # Fully connected layer - the hidden layers are 28 * 28 hence and pooling reduces it by 2
        # Hence, 14 * 14 and mult that with 32 feature/conv layers.
        self.nn_layer = nn.Linear(32 * 14 * 14, 10)
        self.activation = activation_fn  # Activation function
        self.pool = nn.MaxPool2d(2, 2)  # Max pooling layer
    
    def forward(self, x):
        x = self.pool(self.activation(self.conv_layer(x))) 
        x = torch.flatten(x, 1)  # Flatten the feature maps, as the next layer is 1d layer
        x = self.nn_layer(x)  # Output layer
        return x

# ANN-1 excercise
def train_model(model, optimizer, error_func, device, epochs=10, log_dir=""):
    writer = SummaryWriter(log_dir)
    model.to(device)
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        
        for images, labels in trainloader:
            optimizer.zero_grad()  # Reset gradients
            outputs = model(images)  # Forward pass
            loss = error_func(outputs, labels)  # Compute loss
            loss.backward()  # Backpropagation
            optimizer.step()  # Update weights
            
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)  # Get predictions
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        
        train_acc = 100 * correct / total  # Compute accuracy
        writer.add_scalar('Loss/train', running_loss / len(trainloader), epoch)  # Log loss
        writer.add_scalar('Accuracy/train', train_acc, epoch)  # Log accuracy
        print(f'Epoch {epoch+1}, Loss: {running_loss / len(trainloader)}, Accuracy: {train_acc}%')
    writer.close()

# Train model with LeakyReLU and SGD optimizer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"The device I have is {device}")

model1 = SimpleCNN(nn.LeakyReLU())
optimizer1 = optim.SGD(model1.parameters(), lr=0.0001)
err_func = nn.CrossEntropyLoss()
train_model(model1, optimizer1, err_func,device, 10, log_dir='logs/leakyrelu_sgd')

# Train model with LeakyReLU and Adam optimizer
model2 = SimpleCNN(nn.LeakyReLU())
optimizer2 = optim.Adam(model2.parameters())
train_model(model2, optimizer2, err_func,device, 10, log_dir='logs/leakyrelu_adam')

# Train model with Tanh and Adam optimizer
model3 = SimpleCNN(nn.Tanh())
optimizer3 = optim.Adam(model3.parameters())
train_model(model3, optimizer3, err_func,device, 10, log_dir='logs/tanh_adam')



The device I have is cpu
Epoch 1, Loss: 2.2960551202754536, Accuracy: 11.544%
Epoch 2, Loss: 2.2756778196910457, Accuracy: 15.356%
Epoch 3, Loss: 2.2573082694007307, Accuracy: 19.712%
Epoch 4, Loss: 2.2386451304111334, Accuracy: 21.672%
Epoch 5, Loss: 2.219461163596424, Accuracy: 24.166%
Epoch 6, Loss: 2.1996630414977405, Accuracy: 25.916%
Epoch 7, Loss: 2.179634053993713, Accuracy: 27.142%
Epoch 8, Loss: 2.159521300774401, Accuracy: 27.908%
Epoch 9, Loss: 2.13949927100745, Accuracy: 28.494%
Epoch 10, Loss: 2.1200328382384748, Accuracy: 29.214%
Epoch 1, Loss: 1.5120748260137065, Accuracy: 46.878%
Epoch 2, Loss: 1.2330693112462379, Accuracy: 57.264%
Epoch 3, Loss: 1.1292898073373243, Accuracy: 61.074%
Epoch 4, Loss: 1.0620301227130549, Accuracy: 63.5%
Epoch 5, Loss: 1.0138504769643555, Accuracy: 65.21%
Epoch 6, Loss: 0.9711441288671225, Accuracy: 66.576%
Epoch 7, Loss: 0.9428727164140442, Accuracy: 67.524%
Epoch 8, Loss: 0.9141376557405038, Accuracy: 68.636%
Epoch 9, Loss: 0.88440906182

In [8]:
# 2. Fine-tuning the AlexNet model for CIFAR-10
alexnet = models.alexnet(pretrained=False)  # We will initialize AlexNet without pre-trained weights
alexnet.classifier[6] = nn.Linear(in_features=4096, out_features=10) 
optimizer_alexnet = optim.Adam(alexnet.parameters())

train_model(alexnet, optimizer_alexnet, err_func,device, 10, log_dir='logs/alexnet_finetuning')

alexnet_pretrained = models.alexnet(pretrained=True)


alexnet_pretrained.classifier[6] = nn.Linear(in_features=4096, out_features=10)
optimizer_pretrained = optim.Adam(alexnet_pretrained.classifier[6].parameters())

train_model(alexnet_pretrained, optimizer_pretrained, err_func,device, 10, log_dir='logs/alexnet_feature_extraction')

d:\LTU\AdvancedNN\ann_lab_0\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\LTU\AdvancedNN\ann_lab_0\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


RuntimeError: Given input size: (256x1x1). Calculated output size: (256x0x0). Output size is too small